# 株分析のプログラム集

## 1.株価を取得する、

In [5]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import datetime
import os

#チャートグラフ関係
from math import pi
from bokeh.plotting import figure  ,output_notebook , show, output_file
from bokeh.layouts import column
from bokeh.models import LinearAxis, Range1d

#銘柄コード
securitycode='8058'
#開始日
startdate='1/1/2010'#mm/dd/YYYY
#終了日(今日)
enddate=datetime.date.today().strftime('%m/%d/%Y')
#株価データcsvファイルが存在するか確認する。
#ファイルパス
FilePath_CSV='env/株価_'+securitycode+'.csv'
if os.path.isfile(FilePath_CSV):
    df=pd.read_csv(FilePath_CSV)
    date_stock=datetime.date.today()
    #今日が土日の場合は直近の木曜日にする。平日の場合は前日にする。
    if date_stock.weekday()>=5:
        date_stock=datetime.date.today()-datetime.timedelta(days=date_stock.weekday()-3)
    else:
        date_stock=datetime.date.today()-datetime.timedelta(days=1)
    df['Date'] = pd.to_datetime(df['Date'])
    df2=df[df["Date"] >= datetime.datetime(date_stock.year,date_stock.month,date_stock.day)]
    #すでに株価データが存在している場合
    if len(df2)>=1:
        print('yes')
        pdr = df.set_index('Date')
    else:#株価データがない場合
        #株価を取得する（日足）
        pdr =pdr.stooq.StooqDailyReader(symbols=securitycode+'.jp'
                                        , start=startdate, end=enddate).read().sort_values(by='Date',ascending=True)
        #株価を保存する。
        pdr.to_csv(FilePath_CSV)
else:#株価データがない場合
    #株価を取得する（日足）
    pdr =pdr.stooq.StooqDailyReader(symbols=securitycode+'.jp'
                                    , start=startdate, end=enddate).read().sort_values(by='Date',ascending=True)
    #株価を保存する。
    pdr.to_csv(FilePath_CSV)
    
#print(pdr)
#移動平均を計算する
#50移動平均,100移動平均,200移動平均
MA_list=[50,100,200]
for MA in MA_list:
    pdr['MA'+str(MA)+'d']=pdr['Close'].rolling(MA).mean()
#移動平均乖離率
pdr['KairiMA'+str(MA_list[0])+'d'] = (pdr['Close'] - pdr['MA'+str(MA_list[0])+'d'])/pdr['MA'+str(MA_list[0])+'d']*100

#ボリンジャーバンド2シグマを計算する。
BBspan=20
#ボリンジャーバンドを計算
pdr['BBpluse']=pdr['Close'].rolling(BBspan).mean()+pdr['Close'].rolling(BBspan).std()*2
pdr['BBminus']=pdr['Close'].rolling(BBspan).mean()-pdr['Close'].rolling(BBspan).std()*2

#RSIを計算する
rsi_kikan=14
pdr['rsi_up']=pdr['Close'].diff()
pdr['rsi_down']=-1*pdr['rsi_up']
pdr.rsi_up[pdr.rsi_up < 0]=0
pdr.rsi_down[pdr.rsi_down < 0]=0
pdr['rsi']=pdr.rsi_up.rolling(rsi_kikan).mean()/(pdr.rsi_up.rolling(rsi_kikan).mean()
                                              +pdr.rsi_down.rolling(rsi_kikan).mean())*100
#MACDを計算する。
EMA_list=[8,18]
Signal=6
for EMA in EMA_list:
    pdr['EMA'+str(EMA)]=pdr['Close'].ewm(span=EMA).mean()
pdr['MACD'] = pdr['EMA'+str(EMA_list[0])] - pdr['EMA'+str(EMA_list[1])]
pdr['MACD_signal'] = pdr['MACD'].ewm(span=Signal).mean()
pdr['MACD_diff'] = pdr['MACD']-pdr['MACD_signal']

#移動相関係数を計算する。

print(type(pdr))
print(pdr.columns)
print(pdr)
#ローソク足のチャートを作成する。
inc = pdr.Close > pdr.Open
dec = pdr.Open > pdr.Close
w = 12*60*60*1000 # half day in ms
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p = figure(x_axis_type='datetime', tools=TOOLS, plot_width=1000, title = securitycode+' Candlestick')
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3
p.segment(pdr.index, pdr.High, pdr.index, pdr.Low, color="black")
p.vbar(pdr.index[inc], w, pdr.Open[inc], pdr.Close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(pdr.index[dec], w, pdr.Open[dec], pdr.Close[dec], fill_color="#F2583E", line_color="black")
p.line(pdr.index,pdr['MA'+str(MA_list[0])+'d'] ,color = 'red',legend = str(MA_list[0])+'日移動平均')
p.line(pdr.index,pdr['MA'+str(MA_list[1])+'d'] ,color = 'green',legend = str(MA_list[1])+'日移動平均')
p.line(pdr.index,pdr['MA'+str(MA_list[2])+'d'] ,color = 'blue',legend = str(MA_list[2])+'日移動平均')
p.line(pdr.index,pdr.BBpluse ,color = 'purple',legend = 'ボリンジャーバンド+')
p.line(pdr.index,pdr.BBminus ,color = 'purple',legend = 'ボリンジャーバンド-')
p.legend.location = 'top_left'

# 移動平均乖離率のグラフ化    
p1 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
p1.line(pdr.index,pdr['KairiMA'+str(MA_list[0])+'d'] ,color = 'green' ,legend = str(MA_list[0])+'日移動平均乖離率')
p1.line(pdr.index,0,color = 'red',line_width = 2)
# 判例ラベルの位置設定
p1.legend.location = 'top_left'

#RSIのグラフ化
p2 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
p2.line(pdr.index,pdr['rsi'] ,color = 'sandybrown' ,legend = 'RSI('+str(rsi_kikan)+')')
p2.line(pdr.index,70,color = 'red',line_width = 1)
p2.line(pdr.index,50,color = 'lightsteelblue',line_width = 1)
p2.line(pdr.index,30,color = 'red',line_width = 1)
# 判例ラベルの位置設定
p2.legend.location = 'top_left'

#MACDのグラフ化
p3 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
p3.line(pdr.index,pdr['MACD'] ,color = 'green' ,legend = 'MACD('+str(EMA_list[0])+'-'+str(EMA_list[1])+')')
p3.line(pdr.index,pdr['MACD_signal'] ,color = 'red' ,legend = 'Signal('+str(Signal)+')')
# 判例ラベルの位置設定
p3.legend.location = 'top_left'

#volumeの棒グラフを作成する
p4 = figure(width = 1000 , height = 200,x_axis_type = 'datetime' , tools=TOOLS ,x_range = p.x_range)
p4.vbar(pdr.index[inc], w, pdr.Volume[inc] , fill_color="#D5E1DD", line_color="green",legend ='VolumeInc')
p4.vbar(pdr.index[dec], w, pdr.Volume[dec], fill_color="#F2583E", line_color="red",legend ='VolumeDec')
# 判例ラベルの位置設定
p4.legend.location = 'top_left'

#ファイルの保存と表示
output_file('./output/stock_'+securitycode+'_bokeh.html', title=securitycode+"株価分析")
show(column(p,p1,p2,p3,p4))  # open a browser


<ipython-input-5-b87bf86be414>:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-b87bf86be414>:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\users\tazoo\appdata\local\programs\python\python38\lib\site-packages\bokeh\util\deprecation.py:75: BokehDeprecationWarning:

'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead

c:\users\tazoo\appdata\local\programs\python\python38\lib\site-packages\bokeh\util\deprecation.py:75: BokehDeprecationWarning:

'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' 

<class 'pandas.core.frame.DataFrame'>
Index(['Open', 'High', 'Low', 'Close', 'Volume', 'MA50d', 'MA100d', 'MA200d',
       'KairiMA50d', 'BBpluse', 'BBminus', 'rsi_up', 'rsi_down', 'rsi', 'EMA8',
       'EMA18', 'MACD', 'MACD_signal', 'MACD_diff'],
      dtype='object')
               Open     High      Low    Close    Volume      MA50d  \
Date                                                                  
2010-01-04  1885.82  1894.74  1879.31  1880.14   4852651        NaN   
2010-01-05  1930.42  1966.12  1923.94  1927.18  14594324        NaN   
2010-01-06  1946.65  1987.20  1932.86  1969.35  14523925        NaN   
2010-01-07  1997.75  2006.67  1957.18  1968.54  14382882        NaN   
2010-01-08  1954.76  1984.77  1942.61  1970.98  12940525        NaN   
...             ...      ...      ...      ...       ...        ...   
2021-04-22  3022.00  3030.00  2993.00  3013.00   2214800  3038.1608   
2021-04-23  2996.50  3008.00  2988.00  3008.00   2437500  3042.2322   
2021-04-26  3021.00

c:\users\tazoo\appdata\local\programs\python\python38\lib\site-packages\bokeh\util\deprecation.py:75: BokehDeprecationWarning:

'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead

c:\users\tazoo\appdata\local\programs\python\python38\lib\site-packages\bokeh\util\deprecation.py:75: BokehDeprecationWarning:

'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead

c:\users\tazoo\appdata\local\programs\python\python38\lib\site-packages\bokeh\util\deprecation.py:75: BokehDeprecationWarning:

'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead

c:\users\tazoo\appdata\local\programs\python\python38\lib\site-packages\bokeh\util\deprecation.py:75: BokehDeprecationWarning:

'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead

c:\users\tazoo\appdata\local\program